In [13]:
### Initialize
import XLSX, ExcelFiles, DataTables, DataFrames, CSV, Plots

LoadError: [91mArgumentError: Package Plots not found in current path:[39m
[91m- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.[39m


In [6]:
##Reading in Data
using CSV, DataFrames
Data = CSV.read("Data.csv", DataFrame, normalizenames=true)


,hour,heating_kW_th_h_m2_,PV_electricity_kW,Wind_electricity_kW,Demand_kW
,Float64?,Float64?,Float64?,Float64?,Float64?
1,1.0,0.0,0.0,0.0839,1664.74
2,2.0,0.0001,0.0,0.0839,1551.21
3,3.0,0.0,0.0,0.0839,1429.02
4,4.0,0.0012,0.0,0.0839,1350.95
5,5.0,0.0081,0.0,0.0839,1324.06
6,6.0,0.009,0.0,0.0839,1296.71
7,7.0,0.0139,0.0,0.0839,1597.59
8,8.0,0.0194,0.0,0.1533,1479.02
9,9.0,0.0171,0.0,0.1533,1791.12


In [20]:
ElecDemand = Data[1:8760,5] #kW
PV = Data[1:8760,3] #kW
Cost_Elec = Data[1:8760,6]

8760-element Array{Union{Missing, Float64},1}:
 0.0689
 0.065
 0.0603
 0.05
 0.0477
 0.048
 0.0498
 0.0551
 0.0448
 0.0496
 0.0554
 0.0626
 0.0574
 ⋮
 0.0386
 0.0379
 0.038
 0.0396
 0.0423
 0.0461
 0.0477
 0.046
 0.0422
 0.0397
 0.0389
 0.0372

Long Term Variables (Investment Variables) :
E_Rated = Battery Rated Capacity
P_Rated = Battery Rated Power
Binary 1: Invest in Battery
Binary 2: Don't Invest in Battery

Short Term Variables (Operational Variables):
EState_Batt = Battery State of Charge at Time t
PState_Batt = Battery Power at Time t
P_Grid = Power Consumed/ Injected to the grid

Parameters

Cost_BattE = Investment Cost of Battery, energy component

Cost_BattP = Investment cost of battery, power 

Cost_OM= Fixed yearly operation and maintenance costs

DiscountRate = discount rate, werighted average cost of capital

Cost_Electricity = Price of Electricity (optional binary)


In [26]:
using JuMP, Gurobi

T = 8760
model = Model(Gurobi.Optimizer)

# ##Long Term Decision Variables
# @variable(model, E_Rated) #Battery Rated Capacity
# @variable(model, P_Rated) # Battery Rated Power

E_Rated = 10
P_Rated = 10

## Short Term Decision Variables
# @variable(model, EState_Batt[1:T])
# @variable(model, PState_Batt[1:T])
@variable(model, P_Batt[1:T])
@variable(model, P_Grid[1:T])

for t in 1:8760
    ElecDemand[t]=PV[t]+ P_Batt[t]+P_Grid[t]
#     DemandP = ElecDemand[t]  #Demand at hour t
#     PVSupplyP = PV[t]
#     InstPrice = Cost_Elec[t]
#     ElecDemand[t]*3600 = PVSupplyP[t]*3600 + EState_Battery[t] + P_Grid[t]*3600
#     GridDrawPush[t]= P_Grid
#     CostArray[t]= P_Grid * InstPrice #Saves costs in an array
#     EState_Batt[t] =EState_Batt[t-1]+ BatteryP*3600
end

Cost = sum(P_Grid)

@objective(ed, Min, Cost) ##minimize the costs 
# @constraints(model, EState_Batt[i] <=E_Rated)
# @constraints(model, EState_Batt[i] <= 0 )



Academic license - for non-commercial use only - expires 2021-08-04


LoadError: [91mMethodError: [0mCannot `convert` an object of type [92mGenericAffExpr{Float64,VariableRef}[39m[0m to an object of type [91mFloat64[39m[39m
[91m[0mClosest candidates are:[39m
[91m[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at number.jl:6[39m
[91m[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at number.jl:7[39m
[91m[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:250[39m
[91m[0m  ...[39m

In [8]:
Cost_BattE = 300
#300 euro / kWh from Excel Sheet
Cost_BattP = 300
#300 euro/ kW from Excel Sheet
Cost_OM = 20.5 * E_Rated
#20.5 euro / kWh / Yr from Excel Sheet
Discount_Rate = .0025 
## 0.25% ? CHECK THIS
Cost_Elec = Data[1:8760,6]
# from commodity prices on excel -- eur/kWh Elec

LoadError: [91mUndefVarError: E_Rated not defined[39m

In [29]:

    # Define the economic dispatch (ED) model
    ed = Model(Gurobi.Optimizer)
    # Define decision variables
    @variable(ed, P_grid[i = 1:8760]) ## power to/from Grid
    @variable(ed, P_Batt[i = 1:8760]) ## power to from Batt
    ElecDemand[i=1:8760] = PV[i=1:8760] + P_grid + P_Batt
    # Define the objective function
    @objective(ed, Min, P_grid*Cost_Elec)
    # Define the constraint on the maximum and minimum power output of each
    # generator
    @constraint(ed, [i = 1:8760], PGrid[i=1:8760] <= 10000) ## maximum
    #@constraint(ed, [i = 1:8760], g[i] >= g_min[i]) ## minimum
    # Define the constraint on the wind power injection
    #@constraint(ed, w <= w_f)
    # Define the power balance constraint
    #@constraint(ed, sum(g) + w == d)
    # Solve statement
    optimize!(ed)
    # return the optimal value of the objective function and its minimizers
    #return value.(g), value(w), w_f - value(w), objective_value(ed)


Academic license - for non-commercial use only - expires 2021-08-04


LoadError: [91mMethodError: no method matching getindex(::Array{Union{Missing, Float64},1}; i=1:8760)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  getindex(::Array, [91m::Int64[39m) at array.jl:809[91m got unsupported keyword argument "i"[39m[39m
[91m[0m  getindex(::Array, [91m::Int64[39m, [91m::Int64[39m, [91m::Int64...[39m) at array.jl:810[91m got unsupported keyword argument "i"[39m[39m
[91m[0m  getindex(::Array, [91m::UnitRange{Int64}[39m) at array.jl:813[91m got unsupported keyword argument "i"[39m[39m
[91m[0m  ...[39m

PowerDifference = Production - Demand 	kW
Energy Difference = Power Difference * 3600

Option 1: NEG -- Pull Power from the Battery
Option 2: NEG -- Buy Pwoer from the Grid
Option 3: POS -- Save Power to the Battery
Option 4: POS -- Sell Power to the Grid

In [ ]:
BatteryChargeHeld = zeros(8760,1)
BatteryChargeHeld[1]=3000
BatteryChargeHeld

Objective Function is to minimize electricity costs
1) Battery Investment Costs
(Battery Rated Capacity * Cost of Battery Energy) + (Battery Rated Power * Cost of Battery Power)

2) Present Value of Operation & Maintenance Costs for Battery
SUM for every year -- the investment horizon
    (Battery Rated Capacity * Fixed Yearly O&M Costs)/ (1+discountrate)^Year
    
3) Residual Value of Battery at the end of horizon Y (Depreciation or Energy???)

4) Costs/ Revenue from Buying & Selling to Market
SUM for every year -- 
    Sum for every hour (1:8760)
        (Price of Electricity * Power to/from Grid) / (1+discountrate)^year
Note: This assumes that electricity is bouht and sold at the same price. 
RUN A BINARY SIMLULATION LATER TO DEAL WITH THIS

In [ ]:
##Objective Function
TimeHorizon=3
PVOM= 0
CostsRevGrid = 0

## 1) Battery Investment Costs
BattInvCosts = E_Rated*Cost_BattE + P_Rated*Cost_BattP

## 2) PV of OM Costs
for n in Y
    PVOM_yr = E_rated*Cost_OM / (1+DiscountRate)^n
    PVOM = PVOM + PVOM_yr
    
end

## 3) Residual Value at the end of horizon

## 4) Costs/Revenue from Buying/Selling to Market
## For every hour, calculate the difference between supply and demand
PowerDifference = PV-ElecDemand
BatteryStart = E_Rated
for i in 8760
    if PowerDifference <0
        BatE= BatE+PowerDifference*3600
        if BatE <0
            P_Grid = BatE/3600
            E_Grid = BatE
            BatE=0
        end
    end
    if PowerDifference >0
        if 
    BatteryEnergy[i] = BatE
    BatteryPower[i]= BatP
    PGrid[i]=P_Grid
    
end


for y in Y
    for z in 8760
        CostsRevGrid_yr = Cost_Elec[z] * P_Grid[z] / (1+DiscountRate)^y
        CostsRevGrid = CostsRevGrid + CostsRevGrid_yr
    end
end

TotalCosts = BattInvCosts + PVOM + CostsRevGrid

@objective(model, Min,TotalCosts )
